# Resume Specialist Using Reflection Technique

In [2]:
import os
from getpass import getpass
os.environ['FIREWORKS_API_KEY'] = getpass('Enter your FIREWORKS API key: ')
os.environ['OPENAI_API_KEY'] = getpass('Enter your OpenAI API Key: ')

Enter your FIREWORKS API key:  ········
Enter your OpenAI API Key:  ········


In [3]:
! pip install -U --quiet  langchain langgraph fireworks-ai openai


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [4]:
from langchain_community.chat_models.fireworks import ChatFireworks
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a resume assistant tasked with writing excellent resumes."
            " Generate the best resume possible for the user's request."
            " If the user provides critique, respond with a revised version of your previous attempts.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
llm = ChatFireworks(
    model="accounts/fireworks/models/mixtral-8x7b-instruct",
    model_kwargs={"max_tokens": 32768},
)
generate = prompt | llm

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.fireworks.ChatFireworks` was deprecated in langchain-community 0.0.26 and will be removed in 0.2. An updated version of the class exists in the langchain-fireworks package and should be used instead. To use it run `pip install -U langchain-fireworks` and import as `from langchain_fireworks import ChatFireworks`.
  warn_deprecated(


In [5]:
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a resume reviewer evaluating a resume submission. Generate critique and recommendations for the user's submission."
            " Provide detailed recommendations.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
reflect = reflection_prompt | llm

In [6]:
from typing import List, Sequence

async def generation_node(state: Sequence[BaseMessage]):
    return await generate.ainvoke({"messages": state})


async def reflection_node(messages: Sequence[BaseMessage]) -> List[BaseMessage]:
    # Other messages we need to adjust
    cls_map = {"ai": HumanMessage, "human": AIMessage}
    # First message is the original user request. We hold it the same for all nodes
    translated = [messages[0]] + [
        cls_map[msg.type](content=msg.content) for msg in messages[1:]
    ]
    res = await reflect.ainvoke({"messages": translated})
    # We treat the output of this as human feedback for the generator
    print(res.content)
    return HumanMessage(content=res.content)


def should_continue(state: List[BaseMessage]):
    print("$$$$ state lenght $$$$$ :",len(state))
    if len(state) > 4:
        # End after 2 iterations
        return END
    return "reflect"


In [7]:
from langgraph.graph import END, MessageGraph
builder = MessageGraph()
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.set_entry_point("generate")

In [8]:
builder.add_conditional_edges("generate", should_continue)
builder.add_edge("reflect", "generate")
graph = builder.compile()

In [10]:
from openai import OpenAI
client = OpenAI()

audio_file = open("docs/resume.m4a", "rb")
resume_transcription = client.audio.transcriptions.create(
  model="whisper-1", 
  file=audio_file, 
  response_format="text"
)
print(resume_transcription)

My name is Joe Smith. I got my PhD degree in computer science from University of Toronto in 2017. After that, I worked for X company as a machine learning engineer. My task was to develop different machine learning models for X project. I also mentored some junior developers as well. I also deployed and monitored the models in production. After that, I joined Y company in 2022 as a senior language model researcher. My task was to conduct research on LLM models and how to fine tune and also augment them with some techniques. I also prototyped some use cases using the available large language tools. About my skills, I am proficient in Python and I also have a good experience working with deep learning frameworks such as PyTorch and TensorFlow. I also have a good experience using Panda and SQL for data manipulation and I am able to work with machine learning libraries such as Scikit-learn. If you want to reach me, my email is joe.smith at example.com



In [13]:
input = HumanMessage(content = f"""Write a resume based on the following information: \n
                                Resume : {resume_transcription} \n
                                """)


async for event in graph.astream(input):
    print("---")

---
$$$$ state lenght $$$$$ : 2
Overall, the resume is well-written and effectively highlights the candidate's skills and experiences. However, here are some suggestions for further improvement:

1. **Quantify achievements:** Wherever possible, it is helpful to include specific numbers or metrics to quantify the candidate's achievements. For example, instead of saying "Developed machine learning models for X project," the candidate could say "Developed and deployed 5 machine learning models for X project, resulting in a 20% increase in accuracy."
2. **Add a professional title:** Adding a professional title at the top of the resume can help to clearly communicate the candidate's area of expertise and career goals. For example, the title "Machine Learning Expert" or "Senior Research Scientist" could be added above the candidate's name.
3. **Customize the resume for each job application:** It is important to tailor the resume to each job application, highlighting the skills and experience

In [14]:
ChatPromptTemplate.from_messages(event[END]).pretty_print()

================================ Human Message =================================

Write a resume based on the following information: 

                                Resume : My name is Joe Smith. I got my PhD degree in computer science from University of Toronto in 2017. After that, I worked for X company as a machine learning engineer. My task was to develop different machine learning models for X project. I also mentored some junior developers as well. I also deployed and monitored the models in production. After that, I joined Y company in 2022 as a senior language model researcher. My task was to conduct research on LLM models and how to fine tune and also augment them with some techniques. I also prototyped some use cases using the available large language tools. About my skills, I am proficient in Python and I also have a good experience working with deep learning frameworks such as PyTorch and TensorFlow. I also have a good experience using Panda and SQL for data manipulation 